In [1]:
import torch
import os
import numpy as np
from tqdm import tqdm
from sklearn.metrics import roc_curve

In [2]:
def get_threshold(tpr, threshold):
    target = 10**(-2)
    for i in range(len(tpr)):
        if tpr[i] <= target and tpr[i+1] > target:
            return threshold[i]

In [3]:
from shutil import copy2

In [4]:
races = ['African','Asian','Caucasian','Indian']
labels = []
dist = []
for race in races:
        with open(f"./lmks/{race}_lmks.csv",'r') as f:
            lines = f.readlines()
        for line in lines:
            arr = line.strip().split(',')
            labels.append(int(arr[3]))
            dist.append(float(arr[2]))

In [5]:
fpr , tpr ,thresholds = roc_curve(labels,dist)
threshold = get_threshold(tpr,thresholds)
threshold

0.9824120998382568

In [6]:
races = ['African','Asian','Caucasian','Indian']
accuracies = {}
for race in races:
        accuracies[f"{race}"] = {'TP':0,'TN':0,'FP':0,'FN':0}
        with open(f"./lmks/{race}_lmks.csv",'r') as f:
            lines = f.readlines()
        dist = []
        labels = []
        for line in lines:
            arr = line.strip().split(',')
            dist.append(float(arr[2]))
            labels.append(int(arr[3]))
            if float(arr[2]) < threshold:
                with open(f"./results/{race}_results.csv",'a') as f:
                    f.write(f"{arr[0]},{arr[1]},0,{arr[3]}\n")
                if int(arr[3]) == 1:
                    accuracies[f"{race}"]['TN'] += 1
                else:
                    accuracies[f"{race}"]['FN'] += 1
            else:
                with open(f"./results/{race}_results.csv",'a') as f:
                    f.write(f"{arr[0]},{arr[1]},1,{arr[3]}\n")
                if int(arr[3]) == 1:
                    accuracies[f"{race}"]['TP'] += 1
                else:
                    accuracies[f"{race}"]['FP'] += 1
        fpr, tpr, thresholds = roc_curve(labels,dist)
        fpr = np.array(fpr)
        tpr = np.array(tpr)
        with open(f"./roc/{race}_roc.npy",'wb') as f:
            np.save(f,fpr)
            np.save(f,tpr)

In [7]:
print(accuracies)

{'African': {'TP': 12, 'TN': 2988, 'FP': 13, 'FN': 2987}, 'Asian': {'TP': 32, 'TN': 2968, 'FP': 14, 'FN': 2986}, 'Caucasian': {'TP': 55, 'TN': 2945, 'FP': 37, 'FN': 2963}, 'Indian': {'TP': 21, 'TN': 2979, 'FP': 26, 'FN': 2973}}


In [8]:
gar = {}
far = {}
scores = {}
for race in races:
        tp = accuracies[f"{race}"]['TP']
        tn = accuracies[f"{race}"]['TN']
        fp = accuracies[f"{race}"]['FP']
        fn = accuracies[f"{race}"]['FN']
        gar[f"{race}"] = 100*tp/(tn+tp)
        far[f"{race}"] = 100*fp/(fn+fp)
        scores[f"{race}"] = 100*(fn+tp)/(fn+fp+tn+tp)

In [9]:
gar

{'African': 0.4,
 'Asian': 1.0666666666666667,
 'Caucasian': 1.8333333333333333,
 'Indian': 0.7}

In [10]:
far

{'African': 0.43333333333333335,
 'Asian': 0.4666666666666667,
 'Caucasian': 1.2333333333333334,
 'Indian': 0.8669556518839613}

In [11]:
scores

{'African': 49.983333333333334,
 'Asian': 50.3,
 'Caucasian': 50.3,
 'Indian': 49.908318053008834}